# Azure extraction validation API Regression Test Notebook

In [ ]:
#%pip install requests
#%pip install azure-storage-blob
#%pip install python-dotenv

In [ ]:
import requests
import json

In [ ]:
# Replace these with your actual values
from dotenv import dotenv_values, load_dotenv
import sys
sys.path.append('../common')

load_dotenv()

secrets = dotenv_values("secrets.env")
API_KEY = secrets.get("API_KEY")
APIM_NAME = secrets.get("APIM_NAME")
region_override = secrets.get("REGION_OVERRIDE")

HEADERS = {
    "Ocp-Apim-Subscription-Key": API_KEY,
    "Content-Type": "application/json",
    "x-kpmg-charge-code": "1",
    "x-kpmg-region-override": region_override
}

### Upload blob to storage account

In [ ]:
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from datetime import datetime, timedelta
import os

# Set variables
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
container_name = "reg-source"
local_file_path = "./data/HandWrittenText.jpg"  # e.g., "./data/report.pdf"
blob_name = os.path.basename(local_file_path)  # Use file name as blob name

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)   
blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

# Upload the file
with open(local_file_path, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)
print(f"Uploaded '{local_file_path}' to container '{container_name}' as blob '{blob_name}'.")

# Generate SAS token for the blob
sas_token = generate_blob_sas(
    account_name=blob_service_client.account_name,
    container_name=container_name,
    blob_name=blob_name,
    account_key=blob_service_client.credential.account_key,
    permission=BlobSasPermissions(read=True),
    expiry=datetime.utcnow() + timedelta(hours=1)  # SAS valid for 1 hour
)

# Generate full SAS URL
blob_url_with_sas = f"https://{blob_service_client.account_name}.blob.core.windows.net/{container_name}/{blob_name}?{sas_token}"
print("SAS URL:", blob_url_with_sas)


### Create Validation Report

In [ ]:
url = f"https://{APIM_NAME}/extraction/validation/validationReports/"

payload = {
  "urlSource": blob_url_with_sas
}

response = requests.post(url, headers=HEADERS, json=payload)
print(response.json())
data = response.json()  # Convert response to a Python dictionary

# Now extract the reportId
reportId = data.get('reportId', '')

print("Report Id :- ", reportId)

assert response.status_code == 200, f"Validation report process got failed"


### Get validation report

In [ ]:
import time

time.sleep(10)
url = f"https://{APIM_NAME}/extraction/validation/validationReports/{reportId}"
print(reportId)
HEADERS = {
    "Ocp-Apim-Subscription-Key": API_KEY,
    "x-kpmg-charge-code": "1",
    "x-kpmg-region-override": region_override
}

response = requests.get(url, headers=HEADERS)

assert response.status_code == 200, f"Get validation report by report id got failed."
result = response.json()
print(response.json())